# Tutorial MySQL, Pandas, Scikit-learn

## Instalación e importación de bibliotecas necesarias para trabajar

In [8]:
# Actualización de pip si es necesaria
#!pip install --upgrade pip

In [9]:
# Biblioteca de machine learning con herramientas útiles
#!pip install scikit-learn

In [10]:
# Instalación de MySQL y el conector para utilizar Python
#!pip install mysql
#!pip install mysql-connector-python
import mysql.connector as sql
import pymysql

In [11]:
# Biblioteca pandas para manejo de tablas de datos
#!pip install pandas
import pandas as pd

In [12]:
#!pip install matplotlib
from matplotlib import pyplot as plt

## Conexión a la base de datos de MySQL utilizando Python
Para establecer una conexión con una base de datos MySQL, necesitas proveer los parámetros de conexión necesarios, como el host, usario, password y nombre de la db que usará. La función `connect()` puede ser usada para crear el objeto de conexión.

In [13]:
db_config = {
    "host": "localhost",
    "user": "root",
    "password": "",
    "database": "datasets_ml"
}

connection = sql.connect(**db_config)
print(connection)

DatabaseError: 2059 (HY000): Authentication plugin 'mysql_native_password' cannot be loaded: dlopen(/Users/gustavogutierrez/miniconda3/envs/Ambiente-Conda/lib/python3.12/site-packages/mysql/vendor/plugin/mysql_native_password.so, 0x0002): tried: '/Users/gustavogutierrez/miniconda3/envs/Ambiente-Conda/lib/python3.12/site-packages/mysql/vendor/plugin/mysql_native_password.so' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/Users/gustavogutierrez/miniconda3/envs/Ambiente-Conda/lib/python3.12/site-packages/mysql/vendor/plugin/mysql

### Creando un Cursor
Los cursores son usados para interactura con la db. Permiten ejecutar consultas SQL así como recuperar los resultados. Para crear un cursor es necesario usar el método `cursor()` sobre la conexión establecida.

In [ ]:
cursor = connection.cursor()

## Ejecutando comandos SQL
Es posible ejecutar consultas SQL utilizando el método `execute()` de un cursor. Después de realizar una consulta, podemos recuperar los resultados usando métodos como `fetchone()`, `fetchall()`, o `fetchmany()`.

### Descripción de la estructura de la tabla

In [ ]:
table_name = "winequality_red"

cursor.execute(f"DESCRIBE {table_name}")

table_description = cursor.fetchall()

for column_description in table_description:
  print(column_description)

### SELECT

In [ ]:
cursor.execute(f"SELECT * FROM {table_name} LIMIT 10")
result = cursor.fetchall()
for row in result:
  print(row)

In [ ]:
cursor.execute(f"SELECT `fixed acidity` FROM {table_name} LIMIT 10")
result = cursor.fetchall()
for row in result:
  print(row)

In [ ]:
cursor.execute(f"SELECT `fixed acidity`,alcohol,density,`free sulfur dioxide`,class FROM {table_name} LIMIT 10")
result = cursor.fetchall()
for row in result:
  print(row)

In [ ]:
cursor.execute(f"SELECT `fixed acidity`,alcohol,density,`free sulfur dioxide`,class FROM {table_name} WHERE class=0 LIMIT 15")
result = cursor.fetchall()
for row in result:
  print(row)

In [ ]:
cursor.execute(f"SELECT alcohol,density,class FROM {table_name} WHERE class=0  ORDER BY alcohol LIMIT 15")
result = cursor.fetchall()
for row in result:
  print(row)

### ALTER TABLE

In [ ]:
cursor.execute(f"ALTER TABLE {table_name} ADD id INT NOT NULL AUTO_INCREMENT primary key")

In [ ]:
cursor.execute(f"SELECT id,alcohol,density,class  FROM {table_name} ORDER BY id LIMIT 15")
result = cursor.fetchall()
for row in result:
  print(row)

In [ ]:
sql = f"INSERT INTO {table_name} (`fixed acidity`,`volatile acidity`,`citric acid`,`residual sugar`,chlorides, `free sulfur dioxide`, `total sulfur dioxide`, density, pH, sulphates,alcohol,class) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s,%s)"
val = (7.4, 0.7, 0.5, 1.9, 0.076, 14, 35, 0.9978, 3.51, 0.42, 9.6, 0)
cursor.execute(sql, val)

### Guardando cambios
Cuando hacemos cambios en la db (insert, update, delete), es necesario realizar un commit para guardar permanentemente dichos cambios usando el método `commit()` de la conexión establecida.

In [ ]:
connection.commit()

print(cursor.rowcount, "Registro existoso.")

In [ ]:
cursor.execute(f"SELECT * FROM {table_name} ORDER BY id DESC LIMIT 5")
result = cursor.fetchall()
for row in result:
  print(row)

### UPDATE

In [ ]:
sql = f"UPDATE {table_name} SET `fixed acidity`= 8.4 WHERE id=1600"
cursor.execute(sql)
connection.commit()
print(cursor.rowcount, "Cambio existoso.")

In [ ]:
cursor.execute(f"SELECT * FROM {table_name} ORDER BY id DESC LIMIT 5")
result = cursor.fetchall()
for row in result:
  print(row)

### DELETE

In [ ]:
sql = f"DELETE FROM {table_name} WHERE id=1600"
cursor.execute(sql)
connection.commit()
print(cursor.rowcount, "Eliminación existosa.")

In [ ]:
cursor.execute(f"SELECT * FROM {table_name} ORDER BY id DESC LIMIT 5")
result = cursor.fetchall()
for row in result:
  print(row)

## Usando la conexión con MySQL para construir DataFrames utilizando Pandas
Alternativamente, podemos cargar el resultado de un query directamente en un DataFrame de Pandas.
Nota que esta operación posiblemente arrojará un warning por el uso de ``mysql.connector`` en lugar de una conexión vía ``SQLAlchemy``, para fines de esta demostracion `connector` es suficiente. En aplicaciones mas complejas, se sugiere usar [SQLAlchemy](https://saturncloud.io/blog/sqlalchemy-pandas-a-comprehensive-guide-to-database-session-management/).

In [ ]:
df = pd.read_sql_query(f'SELECT `fixed acidity`,`volatile acidity`,`citric acid`,`residual sugar`,chlorides, `free sulfur dioxide`, `total sulfur dioxide`, density, pH, sulphates,alcohol, class from {table_name}',
                       con=connection)
df.head()

### Separando las variables independientes y la dependiente

In [ ]:
# con esto tomamos todas las variables excepto 'class' como variables independiente
X = df.drop(['class'], axis = 1)
# con esto tomamos solo a la variable 'class' como variable dependiente
y = df['class']

### Creando el conjunto de prueba y entrenamiento con scikit-learn

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

### Entrenando un clasificador

In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Entrenamos un simple árbol de decisión con los datos de entrenamiento
clf = DecisionTreeClassifier(max_depth=10, min_samples_split=5, min_samples_leaf=1, max_features='sqrt', random_state=0)
clf.fit(X_train, y_train)

In [ ]:
from sklearn import tree

# Visualizamos (parcialmente) el árbol de decisión
plt.figure(figsize=(12,12))
tree.plot_tree(clf, max_depth=1, feature_names=list(clf.feature_names_in_), filled=True, rounded=True)
plt.show()

In [ ]:
predicciones_modelo = clf.predict(X_test) # con esto aplicamos el modelo a los datos de prueba

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics import ConfusionMatrixDisplay

# confusion matrix
cm = confusion_matrix(y_test, predicciones_modelo)
print ("Confusion Matrix : \n", cm)

# Exactitud de modelo
print('Test accuracy = ', accuracy_score(y_test,predicciones_modelo))

### Usando el modelo con datos nuevos
Insertemos un par de observaciones nuevas en la base de datos. Omitiremos un valor para su clase con el fin de simular la recolección de nuevas observaciones.

In [ ]:
sql = f"INSERT INTO {table_name} (`fixed acidity`,`volatile acidity`,`citric acid`,`residual sugar`,chlorides, `free sulfur dioxide`, `total sulfur dioxide`, density, pH, sulphates,alcohol) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
val = [(7.4, 0.7, 0.5, 1.9, 0.076, 14, 35, 0.9978, 3.51, 0.42, 9.6),(7.1, 0.9, 0.6, 2.1, 0.066, 13, 35, 0.9878, 3.53, 0.52, 9.9)]
cursor.executemany(sql, val)

connection.commit()

print(cursor.rowcount, "Registro existoso.")

Para usar nuestro modelo para clasificar observaciones nuevas, podemos seleccionar tuplas sin atributo clase, clasificarlas con nuestro modelo y actualizar la db con las predicciones.

In [ ]:
# Seleccionamos observaciones sin clasificar
df = pd.read_sql(f'SELECT * from {table_name} where class is null', con=connection)
df.head()

In [ ]:
# Seleccionamos features con los que entrenamos el modelo
X_pred = df.drop(['class', 'id'], axis = 1)

# Usamos el modelo entrenado para clasificar nuevas observaciones
predicciones_modelo = clf.predict(X_pred)
print(predicciones_modelo)

In [ ]:
# Incorporamos predicciones a DataFrame
df['class'] = predicciones_modelo

In [ ]:
df.head()

In [ ]:
# Actualizamos clase en base de datos
for index, row in df.iterrows():
    update_query = f"UPDATE {table_name} SET class = %s WHERE id = %s"
    cursor.execute(update_query, (row['class'], row['id']))

connection.commit()

## Cerrando la conexión y el cursor
Es importante recordar cerrar el cursor y la conexión una vez que terminemos de interactuar con la db para liberar recursos.

In [ ]:
cursor.close()
connection.close()